<a href="https://colab.research.google.com/github/nitinsaimajji/voice1/blob/main/UI_page.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/nitinsaimajji/voice1.git

fatal: destination path 'voice1' already exists and is not an empty directory.


In [10]:
!pip install -q streamlit

In [11]:
!pip install python_speech_features

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
## App

In [30]:
%%writefile app.py
import streamlit as st
from pydub import AudioSegment
from tensorflow.keras.models import load_model
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from pydub import AudioSegment
from python_speech_features import mfcc
from time import time
import IPython.display as ipd
import librosa
import librosa.display
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import os
from keras.utils import to_categorical

def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
        if mfccs.shape[1] < max_pad_len:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]
            
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs
    

fulldatasetpath = '/content/voice1/dataset1'

df = pd.read_csv('/content/voice1/wav_check.csv')


features = []


for index, row in df.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath)+'/',str(row["filename"]))
    
    class_label = row["gender"]
    data = extract_features(file_name)
    
    features.append([data, class_label])


featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 





X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 



model = load_model('model.h5')
num_rows = 40
num_columns = 174
num_channels = 1

max_pad_len = 174




def save_uploaded_file(file):
    with open(file.name, "wb") as f:
        f.write(file.getbuffer())
    return file.name

def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict(prediction_feature)
    predicted_class = le.inverse_transform(np.argmax(predicted_vector, axis=-1)) 
    st.subheader("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = predicted_vector
    for i in range(len(predicted_proba_vector[0])): 
        category = le.inverse_transform(np.array([i]))
        st.write(category[0], "\t\t : ",round(predicted_proba_vector[0][i]*100,2),"%")

def main():
    st.title("Gender Prediction App")
    file = st.file_uploader("Upload an MP3 file", type=["mp3"])
    if file:
       
        file_path = save_uploaded_file(file)

        audio_bytes =file.read()
        st.audio(audio_bytes, format='audio/wav')
        path='/content/'+str(file_path)
        input_file = path
        output_file = "/content/streamcheck.wav"

        audio = AudioSegment.from_file(input_file, format="mp3")
        audio.export(output_file, format="wav")

        filename = '/content/streamcheck.wav' 
        print_prediction(filename) 

if __name__ == "__main__":
    main()

Overwriting app.py


In [31]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.338s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [32]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

35.245.149.69


In [33]:
!npx localtunnel --port 8501

npx: installed 22 in 1.698s
your url is: https://nasty-dogs-hide.loca.lt
^C
